In [41]:
import json
import glob
import os
import numpy as np
from pathlib import Path
import pandas as pd

In [12]:
#still need to write code that loops through all patient folders
##but for now, just starting with the data in one folder

base_path = '/Users/shrey2/Downloads/PY16N013' #modify for your file location
response_path = base_path + '/ResponseInfo/CCEP'
z_path = base_path + '/z-scores'

response_files = os.listdir(response_path)
response_files_path = glob.glob(response_path + '/*.json', recursive=True)

z_files = os.listdir(z_path)
z_files_path = glob.glob(z_path + '/*.json', recursive=True)

In [23]:
# extracting info from each response info file
paths = response_files_path

n = 0
for i in range(len(paths)):
    chNames = []
    # load info into python dictionary
    fileInfo = paths[i].split("_")
    stimCh1 = fileInfo[1];
    stimCh2 = fileInfo[2];
    data = json.load(open(paths[i]))
    
    #Get list of channel names
    for key in jsonData["time"]: chNames.append(key)
        
    # loop over each channel, and extract average time series and information about the peaks

    if n < 1:
        avgResp = np.empty((len(paths),len(chNames),len(jsonData['time'][chNames[0]])))
        significant = np.empty((len(paths),len(chNames)))
        n1Zscore = np.empty((len(paths),len(chNames)))
        n2Zscore = np.empty((len(paths),len(chNames)))
        p2Zscore = np.empty((len(paths),len(chNames)))
        n1Latency = np.empty((len(paths),len(chNames)))
        n2Latency = np.empty((len(paths),len(chNames)))
        p2Latency = np.empty((len(paths),len(chNames)))
        flipped = np.empty((len(paths),len(chNames)))
        n += 1
        samplingRate = np.empty((len(paths)))
        window = np.empty((len(paths),2))
        
        
    for j in range(len(chNames)):
        avgResp[i][j] = data['time'][chNames[j]]
        significant[i][j] = data['significant'][chNames[j]]
        n1Zscore[i][j] = data['zscores'][chNames[j]]['n1'][1]
        n2Zscore[i][j] = data['zscores'][chNames[j]]['n2'][1]
        p2Zscore[i][j] = data['zscores'][chNames[j]]['p2'][1]
        n1Latency[i][j] = data['zscores'][chNames[j]]['n1'][0]+data['window'][0]*data["samplingRate"]/1000
        n2Latency[i][j] = data['zscores'][chNames[j]]['n2'][0]+data['window'][0]*data["samplingRate"]/1000
        p2Latency[i][j] = data['zscores'][chNames[j]]['p2'][0]+data['window'][0]*data["samplingRate"]/1000
        flipped[i][j] = data['zscores'][chNames[j]]['flipped'] 
        
    samplingRate[i] = data["samplingRate"]
    window[i] = data['window']
    
    # set the traces of stimulating channels to zero since they only
    # contain stimulating waveforms / artifacts / saturated signals
    #avgResp{k}(find( any(strcmp([split(chNames{k},'_')'],stimCh1{k})) | any(strcmp([split(chNames{k},'_')'],stimCh2{k}))))= 0;    


In [60]:
#creating dataframe X values

df = pd.DataFrame()
df["chNames"] = chNames * len(response_files_path)
#df["avgResp"] = avgResp.flatten()
df["significant"] = significant.flatten()
df["n1Zscore"] = n1Zscore.flatten()
df["n2Zscore"] = n2Zscore.flatten()
df["p2Zscore"] = p2Zscore.flatten()
df["n1Latency"] = n1Latency.flatten()
df["n2Latency"] = n2Latency.flatten()
df["p2Latency"] = p2Latency.flatten()
df["flipped"] = flipped.flatten()
#df["samplingRate"] = samplingRate.flatten()
#df["flipped"] = flipped.flatten()

print("Shape of dataframe: ", df.shape)
print("Number of response files: ", len(response_files_path))
df.head(15)

Shape of dataframe:  (486, 9)
Number of response files:  6


,chNames,significant,n1Zscore,n2Zscore,p2Zscore,n1Latency,n2Latency,p2Latency,flipped
0,AIH1_AIH2,0.0,-2.439152,-1.965044,-1.965044,76.0,201.0,201.0,0.0
1,AIH2_AIH3,0.0,-1.857802,-1.567210,0.747079,25.0,242.0,117.0,0.0
2,AIH3_AIH4,0.0,0.678827,0.676960,-0.601159,21.0,246.0,94.0,1.0
3,REF1_REF3,0.0,0.667555,0.566369,-0.832691,55.0,201.0,129.0,1.0
4,MIH1_MIH2,0.0,0.000000,0.000000,0.000000,-999.0,-999.0,-999.0,0.0
5,MIH2_MIH3,0.0,0.000000,0.000000,0.000000,-999.0,-999.0,-999.0,0.0
6,MIH3_MIH4,1.0,-17.201848,-13.316489,-6.601373,48.0,389.0,232.0,0.0
7,PIH1_PIH2,0.0,1.803226,9.960486,1.368399,101.0,647.0,141.0,1.0
8,PIH2_PIH3,0.0,1.616359,7.083002,0.713858,36.0,391.0,98.0,1.0
9,PIH3_PIH4,0.0,-1.456600,1.047190,2.444638,22.0,679.0,282.0,0.0


In [73]:
df.sort_values(by="chNames").head(15)

,chNames,significant,n1Zscore,n2Zscore,p2Zscore,n1Latency,n2Latency,p2Latency,flipped,outcome
0,AIH1_AIH2,0.0,-2.439152,-1.965044,-1.965044,76.0,201.0,201.0,0.0,0.0
162,AIH1_AIH2,0.0,1.427479,1.693603,1.106894,21.0,201.0,49.0,1.0,0.0
243,AIH1_AIH2,0.0,-1.579243,-2.725374,0.154807,79.0,582.0,418.0,0.0,0.0
324,AIH1_AIH2,0.0,1.343048,3.250285,-1.615824,101.0,429.0,224.0,1.0,0.0
405,AIH1_AIH2,0.0,1.840616,3.440978,0.758190,101.0,234.0,135.0,1.0,0.0
81,AIH1_AIH2,0.0,-0.670193,-2.648867,1.379705,101.0,341.0,256.0,0.0,0.0
325,AIH2_AIH3,1.0,-6.574136,-6.431050,-4.810286,101.0,232.0,175.0,0.0,0.0
82,AIH2_AIH3,0.0,-2.600126,-1.466423,-0.118053,75.0,236.0,200.0,0.0,0.0
244,AIH2_AIH3,0.0,-0.894063,0.059422,0.510046,21.0,226.0,191.0,0.0,0.0
163,AIH2_AIH3,0.0,-1.920591,-4.827170,-1.920591,101.0,201.0,101.0,0.0,0.0


In [66]:
# adding dataframe outcome values

patient = base_path[base_path.index("P"):]
df["outcome"] = np.zeros(df.shape[0])
df["outcome"][df["chNames"]["LDC3-LDC4"]] = 1

KeyError: 'LDC3-LDC4'